In [16]:
#Import Libraries and Set Up Global Variables

import pyautogui
import time
import webbrowser
import tkinter as tk
from tkinter import ttk, messagebox
import qrcode  # To generate QR codes
from PIL import Image, ImageTk

# Your kitchen's WhatsApp number (with country code)
kitchen_phone = "+917088469168"  # Replace with your kitchen's WhatsApp number

# UPI ID for Payment (Example)
upi_id = "your_upi_id@upi"  # Replace with your actual UPI ID (e.g., example@upi)


In [17]:



# Function to send WhatsApp message instantly
def send_whatsapp_message(order_details, total):
    message_body = f"Order Details:\n{order_details}\nTotal: ₹{total}"

    # WhatsApp Web URL with the message to send
    whatsapp_link = f"https://web.whatsapp.com/send?phone={kitchen_phone}&text={message_body}"
    
    # Open WhatsApp Web URL
    webbrowser.open(whatsapp_link)

    # Wait a little bit to ensure WhatsApp Web is open
    time.sleep(5)  # Wait for WhatsApp Web to load

    # Use pyautogui to simulate typing and sending the message
    pyautogui.press('enter')  # Press enter to send the message

    messagebox.showinfo("WhatsApp Message", "Message sent to the kitchen via WhatsApp!")

    # Generate QR code for WhatsApp link
    qr_code = qrcode.make(whatsapp_link)

    # Save the generated QR code as an image
    qr_code_path = "order_qr.png"
    qr_code.save(qr_code_path)

    # Open the QR code image in the tkinter window
    qr_image = Image.open(qr_code_path)
    qr_image.thumbnail((300, 300))  # Increased size for better visibility
    qr_photo = ImageTk.PhotoImage(qr_image)

    qr_label.config(image=qr_photo)
    qr_label.image = qr_photo  # Keep a reference to avoid garbage collection

    messagebox.showinfo("QR Code Generated", "Scan the QR code to send the order to the kitchen!")


In [18]:
# Function to finalize the order and create payment QR
def finalize_order():
    order_details = "\n".join(order_listbox.get(0, tk.END))
    total = total_label.cget("text").split('₹')[1]
    if not order_details:
        messagebox.showerror("Error", "No items in the order.")
        return
    
    payment_method = payment_method_dropdown.get()
    
    if payment_method == "Online":
        # Create payment QR code
        create_payment_qr(total)
    elif payment_method == "Cash":
        messagebox.showinfo("Cash Payment", f"Please pay ₹{total} in cash to the cashier.")
        qr_label.config(image='')  # Clear QR code image
    else:
        messagebox.showerror("Error", "Please select a payment method.")
    
    # Send the WhatsApp message with the order details
    send_whatsapp_message(order_details, total)

    # Reset the order after finalization
    reset_order()


In [19]:
# Function to create UPI Payment QR Code
def create_payment_qr(total):
    # UPI payment link format: "upi://pay?pa=upi_id&pn=payee_name&mc=merchant_code&tid=transaction_id&txn=amount&msg=order_details"
    upi_link = f"upi://pay?pa={upi_id}&pn=PythonCafe&tid=ORDER123&txn={total}&msg=Order%20Payment"

    # Generate a UPI payment QR code
    qr_code = qrcode.make(upi_link)

    # Save the generated QR code as an image
    qr_code_path = "payment_qr.png"
    qr_code.save(qr_code_path)
    
    # Open the QR code image in the tkinter window
    qr_image = Image.open(qr_code_path)
    qr_image.thumbnail((300, 300))  # Increased size for better visibility
    qr_photo = ImageTk.PhotoImage(qr_image)
    
    qr_label.config(image=qr_photo)
    qr_label.image = qr_photo  # Keep a reference to avoid garbage collection

    messagebox.showinfo("Payment QR Code", f"Scan the QR code to make the payment of ₹{total}")


In [20]:
# Tkinter UI setup
root = tk.Tk()
root.title("Python Cafe Management System")
root.state("zoomed")  # Open in full screen mode
root.resizable(False, False)  # Fixed size window

# Styles - Red and Blue Color Combination
root.config(bg="white")  # White background for a clean look

# Frames for layout
header_frame = tk.Frame(root, bg="#D32F2F", height=60)
header_frame.pack(side="top", fill="x")

menu_frame = tk.Frame(root, bg="#BBDEFB", width=400, padx=15, pady=15)
menu_frame.pack(side="left", fill="y")

order_frame = tk.Frame(root, bg="white", padx=20, pady=10)
order_frame.pack(side="left", expand=True, fill="both")

payment_frame = tk.Frame(root, bg="#BBDEFB", width=400, padx=15, pady=15)
payment_frame.pack(side="right", fill="y")

footer_frame = tk.Frame(root, bg="#D32F2F", height=60)
footer_frame.pack(side="bottom", fill="x")

# Header content
header_label = tk.Label(header_frame, text="Welcome to Python Cafe!", font=("Arial", 30, "bold"), bg="#D32F2F", fg="white")
header_label.pack(pady=15)


In [21]:
# Menu content
menu_label = tk.Label(menu_frame, text="Menu", font=("Arial", 22, "bold"), bg="#BBDEFB", fg="#D32F2F")
menu_label.pack(pady=10)

# Pocket-friendly menu items
menu_items = [
    "Coffee - ₹50",
    "Tea - ₹30",
    "Sandwich - ₹70",
    "Cake - ₹80",
    "Juice - ₹60",
    "Cookies - ₹40",
    "Muffins - ₹50",
    "French Fries - ₹40",
    "Veg Burger - ₹60",
    "Pasta - ₹90",
    "Milkshake - ₹100",
    "Cold Coffee - ₹80",
    "Lemonade - ₹40",
    "Chocolate Bar - ₹30",
    "Soup - ₹70"
]

# Dropdown menu
menu_dropdown = ttk.Combobox(menu_frame, font=("Arial", 16), values=menu_items, state="readonly", width=35)
menu_dropdown.pack(pady=10)
menu_dropdown.set("Select an item")  # Default value


In [22]:
# Quantity input with + and - buttons
quantity_frame = tk.Frame(menu_frame, bg="#BBDEFB")
quantity_frame.pack(pady=15)

quantity_label = tk.Label(quantity_frame, text="1", font=("Arial", 16), width=5, bg="white", relief="solid")
quantity_label.pack(side="left", padx=5)

def increase_quantity():
    current_quantity = int(quantity_label["text"])
    quantity_label.config(text=str(current_quantity + 1))

def decrease_quantity():
    current_quantity = int(quantity_label["text"])
    if current_quantity > 1:
        quantity_label.config(text=str(current_quantity - 1))

minus_button = tk.Button(quantity_frame, text="-", font=("Arial", 16), command=decrease_quantity, width=3, bg="#D32F2F", fg="white")
minus_button.pack(side="left", padx=5)

plus_button = tk.Button(quantity_frame, text="+", font=("Arial", 16), command=increase_quantity, width=3, bg="#D32F2F", fg="white")
plus_button.pack(side="left", padx=5)

add_button = tk.Button(menu_frame, text="Add to Order", font=("Arial", 16), bg="#D32F2F", fg="white", command=lambda: add_to_order())
add_button.pack(pady=15)


In [23]:
# Order content (reduced size)
order_label = tk.Label(order_frame, text="Order Summary", font=("Arial", 22, "bold"), bg="white", fg="#D32F2F")
order_label.pack(pady=15)

order_listbox = tk.Listbox(order_frame, font=("Arial", 14), height=6, width=40, bd=2, relief="solid", selectmode=tk.SINGLE, bg="#BBDEFB", fg="#D32F2F")
order_listbox.pack(padx=10, pady=15, fill="both", expand=True)

# Delete Button functionality
def delete_selected_item():
    selected_item_index = order_listbox.curselection()
    if selected_item_index:
        order_listbox.delete(selected_item_index)
        update_total()
    else:
        messagebox.showwarning("No Selection", "Please select an item to delete.")


In [24]:
# Payment content
total_label = tk.Label(payment_frame, text="Total: ₹0", font=("Arial", 22, "bold"), bg="#D32F2F", fg="white")
total_label.pack(pady=20)

# Payment method dropdown (Cash or Online)
payment_method_label = tk.Label(payment_frame, text="Select Payment Method", font=("Arial", 16), bg="#D32F2F", fg="white")
payment_method_label.pack(pady=10)

payment_method_dropdown = ttk.Combobox(payment_frame, font=("Arial", 16), values=["Cash", "Online"], state="readonly", width=15)
payment_method_dropdown.set("Select payment method")  # Default value
payment_method_dropdown.pack(pady=10)

finalize_button = tk.Button(payment_frame, text="Finalize Order", font=("Arial", 18), bg="#D32F2F", fg="white", command=finalize_order)
finalize_button.pack(pady=15)

# QR Code Section (Fixed and Styled)
qr_label = tk.Label(payment_frame, bg="#BBDEFB")
qr_label.pack(pady=20)


In [25]:
# Function to clear order
def reset_order():
    order_listbox.delete(0, tk.END)
    total_label.config(text="Total: ₹0")
    menu_dropdown.set("Select an item")  # Reset dropdown
    quantity_label.config(text="1")
    qr_label.config(image='')  # Clear QR code image

# Function to update total amount
def update_total():
    total = 0
    for item in order_listbox.get(0, tk.END):
        total += int(item.split('= ₹')[1])
    total_label.config(text=f"Total: ₹{total}")


In [26]:
# Adding items to the order list
def add_to_order():
    selected_item = menu_dropdown.get()
    quantity = int(quantity_label["text"])
    
    # Validation
    if not selected_item or selected_item == "Select an item":
        messagebox.showwarning("No Selection", "Please select an item to add to the order.")
        return
    
    price = int(selected_item.split('- ₹')[1])
    total_price = price * quantity
    order_listbox.insert(tk.END, f"{selected_item} x {quantity} = ₹{total_price}")
    
    update_total()
    
    # Reset quantity to 1
    quantity_label.config(text="1")


In [27]:
# Delete Button (appears only after selecting an item)
delete_button = tk.Button(order_frame, text="Delete Selected Item", font=("Arial", 16), bg="#D32F2F", fg="white", command=delete_selected_item)
delete_button.pack(pady=10)


In [28]:
# Run the application
root.mainloop()
